In [5]:
import os
import markdown
import pdfkit
from lxml import etree
from datetime import datetime
import re


migration_fol = "migration_newhaven"
client = "University of New haven"
iteration = "iteration_0"
timestamp = datetime.strftime(datetime.now(), "%Y%m%d")

base_path = f"/Users/kpowell1/Documents/GitHub/_migration-clients/{migration_fol}/iterations/{iteration}/reports"
new_folder = os.path.join(base_path, "pdf_versions", iteration)
if not os.path.exists(new_folder): os.makedirs(new_folder)

In [6]:
md_files = [f.path for f in os.scandir(base_path) if f.name.endswith('md')]

def replace_tags(text, tags):
    for tag in tags:
        text = text.replace(f"<{tag}>", "")
        text = text.replace(f"</{tag}>", "")
    return text

def generate_pdf(fp):
    with open(fp, mode='r', encoding='utf-8') as input_file:
        text = input_file.read()
        gen_stats = "## General statistics"
        text = text.replace(gen_stats, f"\n{gen_stats}")
    f = re.findall(r"##\s[^\w]", text)
    if (f):
        text = text.replace(f[0], "\n## Success rate\n")
    fn = os.path.basename(fp).replace(".md", ".pdf")
    new_fp = os.path.join(new_folder, f"{timestamp}_{fn}")
    new_fp = new_fp.replace("_transform", "_01_transform")
    new_fp = new_fp.replace("_post", "_02_post")
    text = replace_tags(text, ["details", "summary", "b", "string"])
    html = markdown.markdown(text, extensions=['tables'])
    html_tree = etree.ElementTree(etree.fromstring(f"<html>{html}</html>"))
    tags_to_remove=["p"]
    for tag in tags_to_remove:
        tag_arr = html_tree.findall(f'.//{tag}')
        for t in tag_arr:
            parent = t.getparent()
            parent.remove(t)
    html_string = etree.tostring(html_tree).decode('utf-8')
    print(os.path.basename(new_fp))
    pdfkit.from_string(
        html_string, 
        new_fp, 
        options={
            "header-spacing": 7,
            "margin-top":"2cm",
            "header-left": f"{iteration}",
            "header-center":f"[section] ({client})",
            "header-right":"[page]/[toPage]",
            "header-line":True,
            "footer-center":"EBSCO Information Services | kpowell1@ebsco.com | [isodate]",
            "footer-spacing":3,
            "footer-font-size":8,
            "header-font-size":10
        },
        css="report.css"
    )

for f in md_files:
    generate_pdf(f)

20240131_report_01_transform_users.pdf
20240131_report_02_post_users.pdf
